# Pivoteamento parcial e LU

In [9]:
function elim_gauss(A, b; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    c = copy(b)
    for j = 1:n
        for i = j+1:m
            if abs(U[j,j]) < tol
                error("U_{j,j} ≈ 0.0")
            end
            mij = U[i,j]/U[j,j]
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
            c[i] -= mij*c[j]
        end
    end
    return U, c
end

elim_gauss (generic function with 1 method)

In [10]:
A = [0.0 1 1; 2 1 2; -1 -1 4]
b = A*ones(3)

3-element Array{Float64,1}:
 2.0
 5.0
 2.0

In [11]:
det(A)

-11.0

$A$ é não singular, no entanto o método dado não funciona

In [12]:
U, c = elim_gauss(A, b)

LoadError: LoadError: U_{j,j} ≈ 0.0
while loading In[12], in expression starting on line 1

Mas veja que o sistema equivalente abaixo tem solução pelo algoritmo:

In [15]:
A[[1;3],:] = A[[3;1],:]
A

3x3 Array{Float64,2}:
 -1.0  -1.0  4.0
  2.0   1.0  2.0
  0.0   1.0  1.0

In [18]:
b[[1;3]] = b[[3;1]]
b

3-element Array{Float64,1}:
 2.0
 5.0
 2.0

In [19]:
U, c = elim_gauss(A, b)

(
3x3 Array{Float64,2}:
 -1.0  -1.0   4.0
  0.0  -1.0  10.0
  0.0   0.0  11.0,

[2.0,9.0,11.0])

Uma boa estratégia então seria adicionar a operação elementar de troca de lin